<center><img src="./imge/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 22.基于百度语音实时播报识别物体对象

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：22.基于百度语音实时播报识别物体对象.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：基于百度语音实时播报识别物体对象<br>

## 1.pyttsx3 离线版语音引擎

In [1]:
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate',150)
engine.setProperty('voice','english+f2')
text = 'Welcome to Makerobo AI speech recognition system!!'
engine.say(text)
engine.runAndWait()

In [2]:
!aplay makerobo.mp3

Playing raw data 'makerobo.mp3' : Unsigned 8 bit, Rate 8000 Hz, Mono


## 2.基于百度语音实时播放识别物体对象

In [4]:
import cv2
import jetson.inference
import jetson.utils
import time
import numpy as np

In [5]:
speak = True
item = '欢迎使用创乐博AI智能语音识别系统！！'
confidence = 0
itemOld=''

In [6]:
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

frame_img = widgets.Image(format='jpeg', width=1280, height=720)
display(frame_img)

Image(value=b'', format='jpeg', height='720', width='1280')

In [7]:
from aip import AipSpeech
import pygame
import threading # 线程

#这里需要填你自己的id和密钥
APP_ID='16226519'
API_KEY='5KVxQVES4LSja0u2G4y8m1O9'
SECRET_KEY='KhaXYwGLSmQYgnwHkuXKpV9MO2ta0bQ8'
aipSpeech=AipSpeech(APP_ID,API_KEY,SECRET_KEY)

def sayItem():
    global speak
    global item
    while True:
        if speak == True:
            result = aipSpeech.synthesis(text = item,options={'spd':5,'vol':9,'per':0,})
            if not isinstance(result,dict):
                with open('makerobo.mp3','wb') as f:
                    f.write(result)
                    
            else:print(result)
            #我们利用Jetson Nano自带的pygame
            pygame.mixer.init()
            pygame.mixer.music.load('./makerobo.mp3')
            pygame.mixer.music.play()
            speak=False

x = threading.Thread(target=sayItem,daemon=True)
x.start()

pygame 2.0.0 (SDL 2.0.8, python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
dispW=1280
dispH=720
flip=4
        
# Gstreamer code for improvded Raspberry Pi Camera Quality
camSet='nvarguscamerasrc wbmode=3 tnr-mode=2 tnr-strength=1 ee-mode=2 ee-strength=1 ! video/x-raw(memory:NVMM), width=3264, height=2464, format=NV12, framerate=21/1 ! nvvidconv flip-method='+str(flip)+' ! video/x-raw, width='+str(dispW)+', height='+str(dispH)+', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! videobalance contrast=1.5 brightness=-.2 saturation=1.2 ! appsink'
cam=cv2.VideoCapture(camSet)

net=jetson.inference.imageNet('googlenet')
font=cv2.FONT_HERSHEY_SIMPLEX
timeMark= time.time()
fpsFiltered=0

while True:
    ret,frame = cam.read()
    img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGBA).astype(np.float32)
    img = jetson.utils.cudaFromNumpy(img)
    
    if speak == False:
        classID,confidence = net.Classify(img,dispW,dispH)
        if confidence>=.5:
            item=net.GetClassDesc(classID)
            if item!=itemOld:
                speak = True
        if confidence < .5:
            item=''
        itemOld = item
    dt = time.time() - timeMark
    timeMark = time.time()
    fps = 1/dt
    fpsFiltered=.95*fpsFiltered + .05*fps
    cv2.putText(frame,str(round(fpsFiltered,1)) + ' fps '+item+str(round(confidence,2)),(0,30),font,1,(0,0,255),2)
    frame_img.value = bgr8_to_jpeg(frame)